In [ ]:
!pip install pymupdf
import fitz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.4 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import files

# Upload the file
uploaded_files = files.upload()

# Get the name of the uploaded file
uploaded_filename = list(uploaded_files.keys())[0]

# Rename the file
new_filename = 'file.pdf'
os.rename(uploaded_filename, new_filename)

Saving Jindal Steel & Power Ltd.2020.pdf to Jindal Steel & Power Ltd.2020.pdf


In [ ]:
doc = fitz.open("Adani.pdf")
print(doc.page_count)

FileNotFoundError: ignored

In [ ]:
text = ''
for i in range(doc.page_count):
  page = doc.load_page(i)
  text += page.get_text()


#page = doc.load_page(1)
#text = page.get_text()


In [ ]:
print(text)

In [ ]:
import re
import nltk
import string
from nltk.corpus import stopwords

# In case of any corpus are missing
# download all-nltk
nltk.download("stopwords")

stop_words = stopwords.words("english")
#wordnet = WordNetLemmatizer()
def text_preproc(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [ ]:
text = text_preproc(text)

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

tokens = nltk.word_tokenize(text)
tagged = nltk.pos_tag(tokens)
entities = nltk.chunk.ne_chunk(tagged)
for entity in entities:
    if hasattr(entity, 'label') and entity.label() == 'PERSON':
      # Remove the person name from the text
      text = text.replace(' '.join([word for word, tag in entity.leaves()]), '')

In [ ]:
print(text)

In [ ]:
from google.colab import files
upload = files.upload()

In [ ]:
import csv

def calculate_sentiment_score(text):
    text = text.upper()   # converting all text to uppercase as in LM dict all word are in uppercase
    positive_score = 0
    negative_score = 0

    with open('Loughran-McDonald_MasterDictionary_1993-2021.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row   //

        #Iterating in the dictionary
        for row in reader:
            word = row[0].upper()  # Convert dictionary word to uppercase
            negative = int(row[7])  #row 7 in dict is for negative
            positive = int(row[8])   #row 8 in dict is for  Positive

            #iterating in the text whetther that keyword is present in text or not if present we add its positive or negaative col val to scores
            if word in text:  #  case-insensitive matching
                if(positive>0):
                  positive_score += 1
                if(negative>0):
                  negative_score += 1;

    return positive_score, negative_score
# this i have tried for scaling but i have some doubt in it about the logic of scaling.
# def scale_sentiment_score(sentiment_score):
#     min_score = 0  # Minimum sentiment score in the original range
#     max_score = 1  # Maximum sentiment score in the original range

#     scaled_min = 1  # Scaled minimum sentiment score
#     scaled_max = 10  # Scaled maximum sentiment score

#     scaled_score = ((sentiment_score - min_score) / (max_score - min_score)) * (scaled_max - scaled_min) + scaled_min
#     scaled_score = round(scaled_score)

#     return scaled_score

# Example usage
# text = "This is a sample text with positive and negative words."
# text = "Abandonded abdicated Achieve Advances"
positive_score, negative_score = calculate_sentiment_score(text)
print('Scaled Positive Sentiment Score:', positive_score)
print('Scaled Negative Sentiment Score:', negative_score)


Scaled Positive Sentiment Score: 63
Scaled Negative Sentiment Score: 80
